In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report as cr
from sklearn.metrics import accuracy_score
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r'/content/drive/MyDrive/Client_Segmentation/Client_Segmentation.csv')

In [3]:
df.columns

Index(['CustomerID', 'Orders_Last_Month', 'Orders_Last_3_Months',
       'Orders_Last_6_Months', 'Orders_Last_9_Months', 'Orders_Last_12_Months',
       'Sales_Last_Month', 'Sales_Last_3_Months', 'Sales_Last_6_Months',
       'Sales_Last_9_Months', 'Sales_Last_12_Months', 'Frequency_Last_Month',
       'Frequency_Last_3_Months', 'Frequency_Last_6_Months',
       'Frequency_Last_9_Months', 'Frequency_Last_12_Months', 'Cluster',
       'Customer_segment'],
      dtype='object')

In [4]:
df1 = df.copy()

In [5]:
df2 = pd.read_csv(r'/content/drive/MyDrive/Client_Segmentation/Updated_file.csv')
# df2 = df2[df2['CustomerID'] != 15288]

In [6]:
subset = df1[['CustomerID', 'Cluster']]

In [7]:
df2.head()

,InvoiceNo,StockCode,Description,CustomerID,Country,Country_value,Quantity,UnitPrice,Date
0,536365,3438,WHITE HANGING HEART T-LIGHT HOLDER,17850,United Kingdom,1,6,2.55,2010-12-01 08:26:00
1,536365,2739,WHITE METAL LANTERN,17850,United Kingdom,1,6,3.39,2010-12-01 08:26:00
2,536365,2975,CREAM CUPID HEARTS COAT HANGER,17850,United Kingdom,1,8,2.75,2010-12-01 08:26:00
3,536365,2919,KNITTED UNION FLAG HOT WATER BOTTLE,17850,United Kingdom,1,6,3.39,2010-12-01 08:26:00
4,536365,2918,RED WOOLLY HOTTIE WHITE HEART.,17850,United Kingdom,1,6,3.39,2010-12-01 08:26:00


In [8]:
dffinal = pd.merge(df2, subset[['CustomerID', 'Cluster']], on='CustomerID', how='left')

In [9]:
dffinal

,InvoiceNo,StockCode,Description,CustomerID,Country,Country_value,Quantity,UnitPrice,Date,Cluster
0,536365,3438,WHITE HANGING HEART T-LIGHT HOLDER,17850,United Kingdom,1,6,2.55,2010-12-01 08:26:00,1.0
1,536365,2739,WHITE METAL LANTERN,17850,United Kingdom,1,6,3.39,2010-12-01 08:26:00,1.0
2,536365,2975,CREAM CUPID HEARTS COAT HANGER,17850,United Kingdom,1,8,2.75,2010-12-01 08:26:00,1.0
3,536365,2919,KNITTED UNION FLAG HOT WATER BOTTLE,17850,United Kingdom,1,6,3.39,2010-12-01 08:26:00,1.0
4,536365,2918,RED WOOLLY HOTTIE WHITE HEART.,17850,United Kingdom,1,6,3.39,2010-12-01 08:26:00,1.0
...,...,...,...,...,...,...,...,...,...,...
540450,581587,1492,PACK OF 20 SPACEBOY NAPKINS,12680,France,2,12,0.85,2011-12-09 12:50:00,1.0
540451,581587,1768,CHILDREN'S APRON DOLLY GIRL,12680,France,2,6,2.10,2011-12-09 12:50:00,1.0
540452,581587,2110,CHILDRENS CUTLERY DOLLY GIRL,12680,France,2,4,4.15,2011-12-09 12:50:00,1.0
540453,581587,2111,CHILDRENS CUTLERY CIRCUS PARADE,12680,France,2,4,4.15,2011-12-09 12:50:00,1.0


In [10]:
dffinal[dffinal.isnull().any(axis=1)]

,InvoiceNo,StockCode,Description,CustomerID,Country,Country_value,Quantity,UnitPrice,Date,Cluster
1442,536544,799,DECORATIVE ROSE BATHROOM BOTTLE,15288,United Kingdom,1,1,2.51,2010-12-01 14:32:00,NaN
1443,536544,800,DECORATIVE CATS BATHROOM BOTTLE,15288,United Kingdom,1,2,2.51,2010-12-01 14:32:00,NaN
1444,536544,806,POLKADOT RAIN HAT,15288,United Kingdom,1,4,0.85,2010-12-01 14:32:00,NaN
1445,536544,807,RAIN PONCHO RETROSPOT,15288,United Kingdom,1,2,1.66,2010-12-01 14:32:00,NaN
1446,536544,810,VINTAGE SNAP CARDS,15288,United Kingdom,1,9,1.66,2010-12-01 14:32:00,NaN
...,...,...,...,...,...,...,...,...,...,...
540082,581498,3418,JUMBO BAG RED RETROSPOT,15288,United Kingdom,1,5,4.13,2011-12-09 10:26:00,NaN
540083,581498,3419,JUMBO BAG BAROQUE BLACK WHITE,15288,United Kingdom,1,4,4.13,2011-12-09 10:26:00,NaN
540084,581498,3472,LADIES & GENTLEMEN METAL SIGN,15288,United Kingdom,1,1,4.96,2011-12-09 10:26:00,NaN
540085,581498,3494,S/4 CACTI CANDLES,15288,United Kingdom,1,1,10.79,2011-12-09 10:26:00,NaN


In [11]:
dffinal.dropna(subset=['Cluster'])

,InvoiceNo,StockCode,Description,CustomerID,Country,Country_value,Quantity,UnitPrice,Date,Cluster
0,536365,3438,WHITE HANGING HEART T-LIGHT HOLDER,17850,United Kingdom,1,6,2.55,2010-12-01 08:26:00,1.0
1,536365,2739,WHITE METAL LANTERN,17850,United Kingdom,1,6,3.39,2010-12-01 08:26:00,1.0
2,536365,2975,CREAM CUPID HEARTS COAT HANGER,17850,United Kingdom,1,8,2.75,2010-12-01 08:26:00,1.0
3,536365,2919,KNITTED UNION FLAG HOT WATER BOTTLE,17850,United Kingdom,1,6,3.39,2010-12-01 08:26:00,1.0
4,536365,2918,RED WOOLLY HOTTIE WHITE HEART.,17850,United Kingdom,1,6,3.39,2010-12-01 08:26:00,1.0
...,...,...,...,...,...,...,...,...,...,...
540450,581587,1492,PACK OF 20 SPACEBOY NAPKINS,12680,France,2,12,0.85,2011-12-09 12:50:00,1.0
540451,581587,1768,CHILDREN'S APRON DOLLY GIRL,12680,France,2,6,2.10,2011-12-09 12:50:00,1.0
540452,581587,2110,CHILDRENS CUTLERY DOLLY GIRL,12680,France,2,4,4.15,2011-12-09 12:50:00,1.0
540453,581587,2111,CHILDRENS CUTLERY CIRCUS PARADE,12680,France,2,4,4.15,2011-12-09 12:50:00,1.0


In [12]:
dffinal['Cluster'] = dffinal['Cluster'].astype('Int8')

In [13]:
dffinal.to_csv('Customer_Seg_final.csv')

In [14]:
dffinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540455 entries, 0 to 540454
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   InvoiceNo      540455 non-null  object 
 1   StockCode      540455 non-null  int64  
 2   Description    540455 non-null  object 
 3   CustomerID     540455 non-null  int64  
 4   Country        540455 non-null  object 
 5   Country_value  540455 non-null  int64  
 6   Quantity       540455 non-null  int64  
 7   UnitPrice      540455 non-null  float64
 8   Date           540455 non-null  object 
 9   Cluster        406740 non-null  Int8   
dtypes: Int8(1), float64(1), int64(4), object(4)
memory usage: 42.3+ MB


In [15]:
dffinal['Date'] = pd.to_datetime(dffinal['Date'], errors='coerce')
dffinal['Month'] = dffinal['Date'].dt.month
dffinal['Year'] = dffinal['Date'].dt.year
dffinal['Day'] = dffinal['Date'].dt.day

In [16]:
dffinal['recency'] = (dffinal['Date'] - pd.to_datetime(min(dffinal['Date']))).dt.days
dffinal['Frequency'] = dffinal.groupby('CustomerID')['StockCode'].transform('count')
dffinal['TotalSpend'] = dffinal['Quantity'] * dffinal['UnitPrice']
dffinal['MonetaryValue'] = dffinal.groupby('CustomerID')['TotalSpend'].transform('sum')

In [17]:
dffinal.head()

,InvoiceNo,StockCode,Description,CustomerID,Country,Country_value,Quantity,UnitPrice,Date,Cluster,Month,Year,Day,recency,Frequency,TotalSpend,MonetaryValue
0,536365,3438,WHITE HANGING HEART T-LIGHT HOLDER,17850,United Kingdom,1,6,2.55,2010-12-01 08:26:00,1,12,2010,1,0,312,15.30,5493.79
1,536365,2739,WHITE METAL LANTERN,17850,United Kingdom,1,6,3.39,2010-12-01 08:26:00,1,12,2010,1,0,312,20.34,5493.79
2,536365,2975,CREAM CUPID HEARTS COAT HANGER,17850,United Kingdom,1,8,2.75,2010-12-01 08:26:00,1,12,2010,1,0,312,22.00,5493.79
3,536365,2919,KNITTED UNION FLAG HOT WATER BOTTLE,17850,United Kingdom,1,6,3.39,2010-12-01 08:26:00,1,12,2010,1,0,312,20.34,5493.79
4,536365,2918,RED WOOLLY HOTTIE WHITE HEART.,17850,United Kingdom,1,6,3.39,2010-12-01 08:26:00,1,12,2010,1,0,312,20.34,5493.79


In [18]:
dffinal.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'CustomerID', 'Country',
       'Country_value', 'Quantity', 'UnitPrice', 'Date', 'Cluster', 'Month',
       'Year', 'Day', 'recency', 'Frequency', 'TotalSpend', 'MonetaryValue'],
      dtype='object')

In [19]:
dffinal = dffinal.drop_duplicates()

In [20]:
dffinal.isnull().sum()

InvoiceNo             0
StockCode             0
Description           0
CustomerID            0
Country               0
Country_value         0
Quantity              0
UnitPrice             0
Date                  0
Cluster          133672
Month                 0
Year                  0
Day                   0
recency               0
Frequency             0
TotalSpend            0
MonetaryValue         0
dtype: int64

In [21]:
dffinal[dffinal.isna().any(axis=1)]

,InvoiceNo,StockCode,Description,CustomerID,Country,Country_value,Quantity,UnitPrice,Date,Cluster,Month,Year,Day,recency,Frequency,TotalSpend,MonetaryValue
1442,536544,799,DECORATIVE ROSE BATHROOM BOTTLE,15288,United Kingdom,1,1,2.51,2010-12-01 14:32:00,<NA>,12,2010,1,0,133714,2.51,2065177.68
1443,536544,800,DECORATIVE CATS BATHROOM BOTTLE,15288,United Kingdom,1,2,2.51,2010-12-01 14:32:00,<NA>,12,2010,1,0,133714,5.02,2065177.68
1444,536544,806,POLKADOT RAIN HAT,15288,United Kingdom,1,4,0.85,2010-12-01 14:32:00,<NA>,12,2010,1,0,133714,3.40,2065177.68
1445,536544,807,RAIN PONCHO RETROSPOT,15288,United Kingdom,1,2,1.66,2010-12-01 14:32:00,<NA>,12,2010,1,0,133714,3.32,2065177.68
1446,536544,810,VINTAGE SNAP CARDS,15288,United Kingdom,1,9,1.66,2010-12-01 14:32:00,<NA>,12,2010,1,0,133714,14.94,2065177.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540082,581498,3418,JUMBO BAG RED RETROSPOT,15288,United Kingdom,1,5,4.13,2011-12-09 10:26:00,<NA>,12,2011,9,373,133714,20.65,2065177.68
540083,581498,3419,JUMBO BAG BAROQUE BLACK WHITE,15288,United Kingdom,1,4,4.13,2011-12-09 10:26:00,<NA>,12,2011,9,373,133714,16.52,2065177.68
540084,581498,3472,LADIES & GENTLEMEN METAL SIGN,15288,United Kingdom,1,1,4.96,2011-12-09 10:26:00,<NA>,12,2011,9,373,133714,4.96,2065177.68
540085,581498,3494,S/4 CACTI CANDLES,15288,United Kingdom,1,1,10.79,2011-12-09 10:26:00,<NA>,12,2011,9,373,133714,10.79,2065177.68


In [22]:
dffinal = dffinal[dffinal['Cluster'] != '<NA>']

In [23]:
dffinal.isnull().sum()

InvoiceNo        0
StockCode        0
Description      0
CustomerID       0
Country          0
Country_value    0
Quantity         0
UnitPrice        0
Date             0
Cluster          0
Month            0
Year             0
Day              0
recency          0
Frequency        0
TotalSpend       0
MonetaryValue    0
dtype: int64

In [24]:
dffinal[dffinal.isna().any(axis=1)]

,InvoiceNo,StockCode,Description,CustomerID,Country,Country_value,Quantity,UnitPrice,Date,Cluster,Month,Year,Day,recency,Frequency,TotalSpend,MonetaryValue


In [25]:
x = dffinal.drop(columns=['InvoiceNo', 'Description', 'Country', 'Date', 'Day', 'Month', 'Year', 'Cluster', 'CustomerID', 'StockCode'], axis=1)
y = dffinal.Cluster

In [26]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401515 entries, 0 to 540454
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Country_value  401515 non-null  int64  
 1   Quantity       401515 non-null  int64  
 2   UnitPrice      401515 non-null  float64
 3   recency        401515 non-null  int64  
 4   Frequency      401515 non-null  int64  
 5   TotalSpend     401515 non-null  float64
 6   MonetaryValue  401515 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 24.5 MB


In [27]:
scaler = StandardScaler()
x_scale = scaler.fit_transform(x)

In [28]:
len(x_scale), len(y)

(401515, 401515)

In [29]:
xtrain, xtest, ytrain, ytest = tts(x_scale, y, test_size=0.2, random_state=10)

In [30]:
len(xtrain), len(ytrain), len(xtest), len(ytest)

(321212, 321212, 80303, 80303)

In [31]:
smote = SMOTE()
xtrain_smote, ytrain_smote = smote.fit_resample(xtrain, ytrain)

In [32]:
len(xtrain_smote), len(ytrain_smote)

(963231, 963231)

In [33]:
from xgboost import XGBClassifier

In [34]:
xgb = XGBClassifier()
xgb.fit(xtrain_smote, ytrain_smote)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [35]:
ypred = xgb.predict(xtest)

In [37]:
accuracy_score(ytest, ypred)

0.9999626414953364

In [38]:
# with open('custseg_pred_xgb.pkl', 'wb') as file:
#     pickle.dump(xgb, file)

In [39]:
param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.1, 0.01, 0.001]
    }

In [40]:
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)


In [41]:
grid_search.fit(xtrain_smote, ytrain_smote)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     objective='multi:softprob', ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01, 0.001],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300]},
             verbose=2)

In [42]:
best_params = grid_search.best_params_

In [43]:
best_params

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}

In [44]:
xgb = XGBClassifier(learning_rate = 0.1, max_depth = 5, n_estimators = 200)
xgb.fit(xtrain_smote, ytrain_smote)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [45]:
ypred = xgb.predict(xtest)

In [46]:
accuracy_score(ytest, ypred)

0.9999626414953364

In [47]:
from sklearn.ensemble import RandomForestClassifier

In [48]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'criterion': ['gini', 'entropy']
}

# Choose a hyperparameter tuning method (adjust parameters as needed)
search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')


In [ ]:
search.fit(xtrain_smote, ytrain_smote)